In [1]:
import pandas as pd
import pymysql

# Define the path to the CSV files
csv_files = [
    r"C:\Users\DELL\Desktop\RB\ap_bus_details.csv", 
    r"C:\Users\DELL\Desktop\RB\assam_bus_details.csv", 
    r"C:\Users\DELL\Desktop\RB\chandigarh_bus_details.csv", 
    r"C:\Users\DELL\Desktop\RB\himachal_bus_details.csv",
    r"C:\Users\DELL\Desktop\RB\kerala_bus_details.csv", 
    r"C:\Users\DELL\Desktop\RB\rajasthan_bus_details.csv", 
    r"C:\Users\DELL\Desktop\RB\telangana_bus_details.csv", 
    r"C:\Users\DELL\Desktop\RB\up_bus_details.csv", 
    r"C:\Users\DELL\Desktop\RB\wbtc_bus_details.csv", 
    r"C:\Users\DELL\Desktop\RB\jk_bus_details.csv"
]

# Step 1: Read each CSV file into a DataFrame and store it in a list
df_list = []
for file in csv_files:
    try:
        df_list.append(pd.read_csv(file))
    except FileNotFoundError:
        print(f"File not found: {file}")

# Step 2: Concatenate all DataFrames in the list into a single DataFrame
combined_df = pd.concat(df_list, ignore_index=True)

# Step 3: Add an ID column
combined_df.insert(0, 'id', range(1, len(combined_df) + 1))

# Step 4: Clean and transform the data
# Remove 'INR ' from the Price column
combined_df['Price'] = combined_df['Price'].str.replace('INR ', '', regex=False)

# Extract digits from Seat_Availability
combined_df['Seat_Availability'] = combined_df['Seat_Availability'].str.extract(r'(\d+)')

# Step 5: Save the combined DataFrame to a CSV file
combined_df.to_csv(r"C:\Users\DELL\Desktop\RB\bus_routes.csv", index=False)

# Step 6: Connect to MySQL and create the table
myconnection = pymysql.connect(host='localhost', user='root', passwd='guvi', database="redbus")
cursor = myconnection.cursor()

# Drop any NaN values from the DataFrame
combined_df = combined_df.dropna()

# Step 7: Define the table schema based on DataFrame column types
columns_and_types = ",".join(
    f"{col} {dtype}" 
    for col, dtype in zip(combined_df.columns, combined_df.dtypes)
)
columns_and_types = columns_and_types.replace("float64", "FLOAT").replace("object", "TEXT").replace("int64", "INT")
table_name = "bus_routes"

# Step 8: Create the table if it doesn't already exist
cursor.execute(f"CREATE TABLE IF NOT EXISTS {table_name} ({columns_and_types})")

# Step 9: Insert data into the MySQL table
for _, row in combined_df.iterrows():
    cursor.execute(f"INSERT INTO {table_name} VALUES {tuple(row)}")
myconnection.commit()

# Step 10: Close the cursor and connection
cursor.close()
myconnection.close()
